## Routing

- #### Logical routing
    A routing mechanism where the LLM chooses the DB based on the question asked.

- #### Semantic routing
    A routing mechanism where the question is embeded, and a prompt is chosen based on the similarity search.

In [ ]:
! pip -q install langchain_community tiktoken langchain-deepseek langchainhub chromadb langchain dotenv bs4 langchain-text-splitters langchain-ollama

In [ ]:
# Setup

import os
from dotenv import load_dotenv
from langchain_ollama import OllamaEmbeddings
from langchain_deepseek import ChatDeepSeek

load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

# Loading my LLM API Key

EMBEDDING_MODEL_NAME = "qwen3-embedding:0.6b"
DEEPSEEK_MODEL_NAME='deepseek-chat'

OLLAMA_EMBEDDING = OllamaEmbeddings(model=EMBEDDING_MODEL_NAME)
DEEPSEEK_LLM = ChatDeepSeek(model=DEEPSEEK_MODEL_NAME, temperature=0, api_key=os.getenv('DEEPSEEK_API_KEY'))


##### Logical Routing

In [ ]:
# Logical routing

from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class RouteQuery(BaseModel):
    """
    Route a user query to the most relevant datasource.
    """
    datasource: Literal['python_docs', 'js_docs', 'golang_docs'] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question"
    )

# LLM with function call
structuredLLM = DEEPSEEK_LLM.with_structured_output(RouteQuery)

systemp_prompt = """
You are an expert at routing a user question to the approoriate data source.

Based on the programming language the question is referring to, route it to the relevant data source.
"""

prompt = ChatPromptTemplate.from_messages(
    [('system', systemp_prompt), ('human', '{question}')]
)

router = prompt | structuredLLM

In [ ]:
question = """
Why doesn't the following code work?
    ---
    def add_function():
        return a + b
    ---
"""

result = router.invoke({'question': question})
result

In [ ]:
def choose_route(result):
    if 'python' in result.datasource.lower():
        return 'chain for python_docs'
    if 'js' in result.datasource.lower():
        return 'chain for js_docs'
    else:
        return 'golang_docs'

from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)

full_chain.invoke({'question': question})
    

##### Semantic Routing

In [ ]:
from langchain_classic.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


physics_template = """
You are a very smart physics professor.\
You are great at answering questions about physics in a concise and easy manner to understand. \
When you don't know the answer to a question you admit that you don't know

Here is a question: {query}
"""

math_template = """
You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question: {query}
"""

embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL_NAME)
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

def prompt_router(input):
    query_embedding = embeddings.embed_query(input['query'])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")

    return PromptTemplate.from_template(most_similar)


chain =(
    {'query': RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | DEEPSEEK_LLM
    | StrOutputParser()
)

print(chain.invoke( "What is the logarithm of negative 2?"))